In [ ]:
!pip -q install redis redis-cli

In [1]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)


## RedisJSON: Storing and Querying JSON Documents in Redis

### JSON Introduction

JSON (JavaScript Object Notation) is a lightweight data-interchange format that represents data objects as attribute–value pairs and arrays. It supports strings, numbers, booleans, null, objects, and arrays. These primitive values are also called JSON scalars.

Example JSON document:

```python
# Example JSON representation of a product
{
  "id": "B085LVV8R7",
  "name": "SENNHEISER Momentum True Wireless 2",
  "tags": ["hifi", "headphones", "wireless"]
}
```

> **Sidenote**
> [JSON Specification | ECMA](https://www.ecma-international.org/publications-and-standards/standards/ecma-404/)
> **Concept**: `JSON` → A standard format for representing structured data.
> **Context**: Widely used in APIs, databases, and configurations.
> **Example**: Objects with key-value pairs and arrays.
> **Implication**: Ensures interoperability and human-readable data exchange.

```python
# RedisJSON usage context
# Store JSON documents as Redis keys and access sub-elements efficiently.
```

### What is RedisJSON?

RedisJSON is a Redis module that implements JSON as a native data type. It allows storing, updating, and retrieving JSON values directly in Redis. Documents are stored as binary tree structures for fast access to sub-elements.

Key features include:

* Full support for JSON standard
* JSONPath syntax for selecting elements
* Atomic operations for all JSON value types
* Fast retrieval and updates on sub-elements

> **Sidenote**
> [RedisJSON | Docs - Redis](https://redis.io/docs/latest/develop/data-types/json/)
> **Concept**: `RedisJSON` → Redis module enabling JSON as a first-class data type.
> **Context**: Turns Redis into a document store.
> **Example**: Store and query structured product catalog entries.
> **Implication**: Enables document-oriented patterns on Redis.

```python
# Example JSON document for Redis storage
{
  "id": "B085LVV8R7",
  "name": "SENNHEISER Momentum True Wireless 2",
  "price": 150,
  "category": {
    "id": "wireless_headphones",
    "name": "Wireless headphones"
  },
  "tags": ["hifi", "headphones", "wireless"]
}
```

### Basic API with RedisJSON

#### Setting JSON Documents

```python
# Store a JSON product document
r.execute_command("JSON.SET", "product:1", "$", '{"id":"B085LVV8R7","name":"SENNHEISER Momentum True Wireless 2","price":150,"category":{"id":"wireless_headphones","name":"Wireless headphones"},"tags":["hifi","headphones","wireless"]}')
print(r.execute_command("JSON.GET", "product:1", "$"))
```

> **Sidenote**
> [JSON.SET | Docs - Redis](https://redis.io/docs/latest/commands/json.set/)
> **Command**: `JSON.SET` → Set a JSON value at a key.
> **Pattern**: `JSON.SET key path value`
> **Example**: `JSON.SET product:1 $ '{"name":"AirPods"}'`
> **Result**: Stores the document under the key.

#### Retrieving JSON Documents

```python
# Retrieve full JSON document
r.execute_command("JSON.GET", "product:1", "$")

# Retrieve only the product name
r.execute_command("JSON.GET", "product:1", "$.name")

# Retrieve nested category name
r.execute_command("JSON.GET", "product:1", "$.category.name")
```

> **Sidenote**
> [JSON.GET | Docs - Redis](https://redis.io/docs/latest/commands/json.get/)
> **Command**: `JSON.GET` → Retrieve JSON values.
> **Pattern**: `JSON.GET key [path]`
> **Example**: `JSON.GET product:1 $.name`
> **Result**: Returns the value of the JSON path.

#### Numeric Operations

```python
# Increase product price by 5
r.execute_command("JSON.NUMINCRBY", "product:1", "$.price", 5)
print(r.execute_command("JSON.GET", "product:1", "$.price"))

# Decrease product price by 5
r.execute_command("JSON.NUMINCRBY", "product:1", "$.price", -5)
print(r.execute_command("JSON.GET", "product:1", "$.price"))
```

> **Sidenote**
> [JSON.NUMINCRBY | Docs - Redis](https://redis.io/docs/latest/commands/json.numincrby/)
> **Command**: `JSON.NUMINCRBY` → Increment or decrement a numeric value.
> **Pattern**: `JSON.NUMINCRBY key path number`
> **Example**: `JSON.NUMINCRBY product:1 $.price 10`
> **Result**: Updates the numeric field by the given value.

#### Array Operations

```python
# Append new tags to product
r.execute_command("JSON.ARRAPPEND", "product:1", "$.tags", '"audio"', '"bluetooth"')
print(r.execute_command("JSON.GET", "product:1", "$.tags"))

# Count number of tags
r.execute_command("JSON.ARRLEN", "product:1", "$.tags")
```

> **Sidenote**
> [JSON.ARRAPPEND | Docs - Redis](https://redis.io/docs/latest/commands/json.arrappend/)
> **Command**: `JSON.ARRAPPEND` → Append values to an array.
> **Pattern**: `JSON.ARRAPPEND key path value [value ...]`
> **Example**: `JSON.ARRAPPEND product:1 $.tags '"newtag"'`
> **Result**: Adds new elements to array.

> **Sidenote**
> [JSON.ARRLEN | Docs - Redis](https://redis.io/docs/latest/commands/json.arrlen/)
> **Command**: `JSON.ARRLEN` → Get length of a JSON array.
> **Pattern**: `JSON.ARRLEN key path`
> **Example**: `JSON.ARRLEN product:1 $.tags`
> **Result**: Returns number of elements in array.

### JSON with RediSearch

RedisJSON integrates with RediSearch to provide indexing, full-text search, and query capabilities over JSON documents.

#### Creating an Index

```python
# Create an index for product documents
r.execute_command("FT.CREATE", "products", "ON", "JSON",
                  "PREFIX", "1", "product:",
                  "SCHEMA",
                  "$.id", "AS", "id", "TEXT",
                  "$.name", "AS", "name", "TEXT", "NOSTEM",
                  "$.price", "AS", "price", "NUMERIC", "SORTABLE",
                  "$.category.id", "AS", "category_id", "TEXT", "NOSTEM",
                  "$.category.name", "AS", "category_name", "TEXT",
                  "$.tags.*", "AS", "tags", "TAG")
print("Index created")
```

> **Sidenote**
> [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/)
> **Command**: `FT.CREATE` → Create a RediSearch index.
> **Pattern**: `FT.CREATE index_name ON JSON PREFIX N schema`
> **Example**: `FT.CREATE products ON JSON PREFIX 1 "product:" SCHEMA $.id AS id TEXT`
> **Result**: Creates an index over JSON fields.

#### Searching with Indexes

```python
# Search by text across fields
r.execute_command("FT.SEARCH", "products", "headphones")

# Search by tag
r.execute_command("FT.SEARCH", "products", "@tags:{wireless}")

# Search by price range
r.execute_command("FT.SEARCH", "products", "@price:[120 150]")
```

> **Sidenote**
> [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/)
> **Command**: `FT.SEARCH` → Query an index.
> **Pattern**: `FT.SEARCH index query`
> **Example**: `FT.SEARCH products "@tags:{wireless}"`
> **Result**: Returns documents matching query.

#### Dropping an Index

```python
# Drop an index when no longer needed
r.execute_command("FT.DROPINDEX", "products")
print("Index dropped")
```

> **Sidenote**
> [FT.DROPINDEX | Docs - Redis](https://redis.io/docs/latest/commands/ft.dropindex/)
> **Command**: `FT.DROPINDEX` → Delete an index.
> **Pattern**: `FT.DROPINDEX index_name`
> **Example**: `FT.DROPINDEX products`
> **Result**: Removes the index definition.

---

## Sources

* [JSON Specification | ECMA](https://www.ecma-international.org/publications-and-standards/standards/ecma-404/)
* [RedisJSON | Docs - Redis](https://redis.io/docs/latest/develop/data-types/json/)
* [JSON.SET | Docs - Redis](https://redis.io/docs/latest/commands/json.set/)
* [JSON.GET | Docs - Redis](https://redis.io/docs/latest/commands/json.get/)
* [JSON.NUMINCRBY | Docs - Redis](https://redis.io/docs/latest/commands/json.numincrby/)
* [JSON.ARRAPPEND | Docs - Redis](https://redis.io/docs/latest/commands/json.arrappend/)
* [JSON.ARRLEN | Docs - Redis](https://redis.io/docs/latest/commands/json.arrlen/)
* [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/)
* [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/)
* [FT.DROPINDEX | Docs - Redis](https://redis.io/docs/latest/commands/ft.dropindex/)




### Original Transcript

Article - Time Series
Time Series Introduction
A Time Series Database System (TSDBS) is a database system (DBS) that is optimized for handling time-series data. RedisTimeSeries is a Redis Module adding a Time Series data structure to Redis. This section will explain the motivation behind Time Series Database Systems (TDBS) by explaining how RedisTimeSeries fits into it.

 

Why Time Series Databases?
Why do we need a special-purpose time-series database system? Here are some alternative solutions that accomplish something similar:

Relational DBS: Using a time column with an appropriate storage structure (i.e. B-Tree)
Column Store: Leveraging a timestamp data type (e.g., timeuuid) and fast insertions via append-only storage structures
Redis: Using sorted sets, where the score is used for storing the timestamp
The answer is that none of these solutions are complete. They end up pushing the complexity of features like retention and downsampling to the application side.

In addition, the query pattern differs from what existing databases were meant to do. Existing databases were meant to have fewer writes than reads. Time-series data typically has a high ingestion rate and a lower number of read-only queries.

Typical Use Cases
IOT & Sensor Monitoring: Devices are emitting a stream of events. A TSDBS allows monitoring and reacting to this data in real-time.
Application Performance/Health Monitoring: Monitor the performance and availability of applications and services.
Real-time Analytics: Process, Analyze and react in real-time (e.g, for selling equities, performing predictive maintenance, product recommendations, or price adjustments)
Running Time Series
You can either run the OSS version of Time Series or create a Redis Enterprise database which has the module deployed. The simplest way to try Time Series is to use the Docker container.

docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack

Creating a time-series
You create a new time-series with the TS.CREATE command:

TS.CREATE key [RETENTION retentionTime] [UNCOMPRESSED] [CHUNK_SIZE size] [DUPLICATE_POLICY policy] [LABELS label value..]

Each time-series is stored under a key in Redis and the key argument above is the key name for the time-series.

Example:

TS.CREATE sensor1 RETENTION 2678400000

This will create a time-series called sensor1 and trim it to values of up to one month.

Adding data points
Adding a new data point to a time-series is straightforward:

TS.ADD key timestamp value [RETENTION retentionTime] [UNCOMPRESSED] [CHUNK_SIZE size] [ON_DUPLICATE policy] [LABELS label value..]

The timestamp argument is the UNIX timestamp of the sample in milliseconds and value is the numeric data value of the sample. Example:

TS.ADD sensor1 1626434637914 26

To add a datapoint with the current timestamp you can use a * instead of a specific timestamp:

TS.ADD sensor1 * 26

You can append data points to multiple time-series at the same time with the TS.MADD command:

TS.MADD key timestamp value [key timestamp value ...]

Additional Time Series features
Besides creating time-series and adding points to them, the RedisTimeSeries module offers several capabilities such as:

Deleting points from a time series
Attaching key-value metadata to data points, so samples can be filtered or grouped
Compacting data with downsampling
Filtering by value, timestamp, and by labels
Combine values of one or more time-series by leveraging aggregation functions